# With Tensorflow

Maybe don't do full tensorflow alt implementation

Instead:
* use a mix of existing code (HF, PT, MLX)
* and use functions in TF, eg, from google/bert

In [ ]:
%load_ext autoreload
%autoreload 2

In [116]:
import tensorflow as tf

import torch
import torch.nn as tnn

import mlx.nn as nn
import mlx.core as mx

from alt.pt.infer import load_model_tokenizer
from utils import preprocess_tokenize_function

# Load

# Train

In [ ]:
model, tokenizer = load_model_tokenizer(device="mps")

inputs = tokenizer(question, context, return_tensors="pt")
inputs.to("mps")

with torch.no_grad():
    outputs = model(**inputs)

inputs.keys(), outputs.keys()

### Outputs

In [139]:
def get_out():
    start_logits = [0.9, 1.9, 0.1, 10.9, 1.5]
    end_logits = [1.9, 0.8, 0.2, 3.3, 11.5]
    start_positions = [5, 1, 3, 11, 2]
    end_positions = [1, 6, 4, 9, 12]
    return start_logits, end_logits, start_positions, end_positions

# Compute loss

with TF, HF/PT, etc

In [105]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

### MLX

In [223]:
start_logits, end_logits, start_positions, end_positions = (
    mx.array(x, dtype=mx.float32) for x in get_out()    
)

In [224]:
start_logits.shape, start_logits

((5,), array([0.9, 1.9, 0.1, 10.9, 1.5], dtype=float32))

In [234]:
def ce(l, p):
    return nn.losses.cross_entropy(l, p, reduction="none").item()

def ce2(l, p):
    return nn.losses.nll_loss(nn.log_softmax(l), p).item()

In [243]:
ce(start_logits, start_positions), ce(end_logits, end_positions)

(-118.69972229003906, -163.69961547851562)

In [245]:
x = nn.log_softmax(start_logits)
x, x.shape

(array([-10.0003, -9.00027, -10.8003, -0.000271797, -9.40027], dtype=float32),
 (5,))

In [303]:
logits = mx.array([[0.2, -2.0], [-1.9, 3.8]])
targets = mx.array([0, 1])

logits = mx.array([0.105361, 0.223144, 1.20397, 0.916291])
targets = mx.array([0, 0, 1, 1])

In [304]:
logits

array([0.105361, 0.223144, 1.20397, 0.916291], dtype=float32)

In [305]:
targets

array([0, 0, 1, 1], dtype=int32)

In [312]:
mx.log(1)

array(0, dtype=float32)

In [309]:
logits[0]

array(0.105361, dtype=float32)

In [306]:
nn.losses.binary_cross_entropy(logits, targets, with_logits=True, reduction="none")

array([0.747215, 0.81093, 0.262365, 0.336472], dtype=float32)

In [313]:
mx.logaddexp(0.0, logits) - logits * targets

array([0.747215, 0.81093, 0.262365, 0.336472], dtype=float32)

In [314]:
nn.losses.cross_entropy(logits, targets, reduction="none")

array(-0.0168054, dtype=float32)

In [276]:
nn.losses.nll_loss(logits, targets)

ValueError: [take_along_axis] Indices of dimension 2 does not match array of dimension 1.

In [259]:
-mx.take_along_axis(start_logits, start_positions[..., None], axis=-1)

ValueError: [take_along_axis] Indices of dimension 2 does not match array of dimension 1.

In [251]:
nn.losses.nll_loss(x[..., None], start_positions)

ValueError: [gather] Got indices with invalid dtype. Indices must be integral.

In [252]:
x.shape, x[..., None].shape

((5,), (5, 1))

In [254]:
x.dtype, x[..., None].dtype

(mlx.core.float32, mlx.core.float32)

In [248]:
start_positions[..., None].shape

(5, 1)

In [242]:
start_positions.shape

(5,)

In [240]:
nn.losses.nll_loss(x, start_positions)

ValueError: [take_along_axis] Indices of dimension 2 does not match array of dimension 1.

In [236]:
ce2(start_logits, start_positions), ce2(end_logits, end_positions)

ValueError: [take_along_axis] Indices of dimension 2 does not match array of dimension 1.

### PT

https://github.com/huggingface/transformers/blob/v4.44.2/src/transformers/models/bert/modeling_bert.py#L1996

such as:

```
total_loss = None
if start_positions is not None and end_positions is not None:
    # If we are on multi-GPU, split add a dimension
    if len(start_positions.size()) > 1:
        start_positions = start_positions.squeeze(-1)
    if len(end_positions.size()) > 1:
        end_positions = end_positions.squeeze(-1)
    # sometimes the start/end positions are outside our model inputs, we ignore these terms
    ignored_index = start_logits.size(1)
    start_positions = start_positions.clamp(0, ignored_index)
    end_positions = end_positions.clamp(0, ignored_index)

    loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss) / 2
```

In [181]:
start_logits, end_logits, start_positions, end_positions = (
    torch.tensor(x, dtype=torch.float32) for x in get_out()    
)

In [182]:
start_logits.shape, start_logits.size(), start_logits

(torch.Size([5]),
 torch.Size([5]),
 tensor([ 0.9000,  1.9000,  0.1000, 10.9000,  1.5000]))

In [189]:
loss_fct = tnn.CrossEntropyLoss(reduction="none") #ignore_index=ignored_index)
loss_fct(start_logits, start_positions), loss_fct(end_logits, end_positions)

(tensor(110.2060), tensor(192.8121))

### TF

from https://github.com/google-research/bert/blob/master/run_squad.py#L646

In [190]:
start_logits, end_logits, start_positions, end_positions = (
   tf.convert_to_tensor(x, dtype=tf.float32) for x in get_out()    
)

start_positions, end_positions = (
   tf.cast(x, dtype=tf.int32) for x in 
    (start_positions, end_positions)
)

# run_squad.py has:
# seq_length = modeling.get_shape_list(input_ids)[1]
seq_length = len(start_logits)

In [191]:
start_logits.shape, start_logits

(TensorShape([5]),
 <tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 0.9,  1.9,  0.1, 10.9,  1.5], dtype=float32)>)

In [192]:
start_positions

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 5,  1,  3, 11,  2], dtype=int32)>

In [193]:
def compute_loss(logits, positions):
    one_hot_positions = tf.one_hot(
        positions, depth=seq_length, dtype=tf.float32)
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    loss = -tf.reduce_mean(
        tf.reduce_sum(one_hot_positions * log_probs, axis=-1))
    return loss

In [194]:
compute_loss(start_logits, start_positions).numpy(), compute_loss(end_logits, end_positions).numpy()

(3.960163, 2.140151)

Check components

In [195]:
tf.one_hot(start_positions, depth=seq_length, dtype=tf.float32)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)>

In [201]:
tf.nn.log_softmax(start_logits, axis=-1).numpy()

array([-1.0000272e+01, -9.0002718e+00, -1.0800271e+01, -2.7187943e-04,
       -9.4002714e+00], dtype=float32)

In [203]:
nn.log_softmax(mx.array(start_logits), axis=-1)

array([-10.0003, -9.00027, -10.8003, -0.000271797, -9.40027], dtype=float32)

In [204]:
-tf.reduce_mean((one_hot_positions * log_probs, axis=-1))

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (1215032111.py, line 1)

array(-118.7, dtype=float32)

In [207]:
nn.losses.cross_entropy(mx.array(start_logits), mx.array(start_positions), reduction='sum')

array(-118.7, dtype=float32)

In [208]:
nn.losses.cross_entropy(mx.array(start_logits), mx.array(start_positions), reduction='mean')

array(-118.7, dtype=float32)

In [209]:
nn.losses.binary_cross_entropy(mx.array(start_logits), mx.array(start_positions))

array(-22.5947, dtype=float32)

In [ ]:
start_positions = features["start_positions"]
end_positions = features["end_positions"]

start_loss = compute_loss(start_logits, start_positions)
end_loss = compute_loss(end_logits, end_positions)

total_loss = (start_loss + end_loss) / 2.0

In [ ]:
def loss_fn(model, input_ids, token_type_ids, attention_mask, start_positions,
            end_positions, reduce=True):
    start_logits, end_logits = model(
        input_ids=input_ids,
        token_type_ids=token_type_ids,
        attention_mask=attention_mask)
    slosses = nn.losses.cross_entropy(start_logits, start_positions)
    elosses = nn.losses.cross_entropy(end_logits, end_positions)
    if reduce:
        slosses = mx.mean(slosses)
        elosses = mx.mean(elosses)
    loss = (slosses + elosses) / 2
    return loss


def loss_from_logits_positions(logits, positions):
    pass